In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC, NuSVC
from sklearn.metrics import f1_score, accuracy_score, make_scorer, roc_curve, classification_report
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import f1_score, accuracy_score, make_scorer
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score,confusion_matrix
import pickle

data=pd.read_csv('DIG.csv')
df=data.copy()
df = df.drop(['Unnamed: 0', 'ID'], axis=1)
df.head()

x = df.drop(['STRK'],axis=1)
y=df['STRK']

xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.2,shuffle=True,random_state=30)

#see data if it is balanced 
#hint is not balanced 

#min max scaling 
# compute statistics on training fold for feature rescaling (min-max)
minima = np.min(xtrain, axis=0)
maxima = np.max(xtrain, axis=0)

# rescale train and test sets
xtrain = (xtrain - minima) / (maxima - minima)
xtest = (xtest - minima) / (maxima - minima)

ytrain.shape

(5440,)

In [4]:
columns1 = x.columns
df_resampling_x = pd.DataFrame(xtrain, columns = columns1)
#print(xtrain)
df_resampling_y = pd.DataFrame(ytrain, columns = ['id', 'STRK'])
del df_resampling_y['id']
concat = [df_resampling_x, df_resampling_y]
resampled_train_df = pd.concat(concat, axis=1)

total = len(resampled_train_df['STRK'])
is_stroke = list(resampled_train_df['STRK']).count(1)
print("NO STROKE: ",total-is_stroke)
print("STROKE: ",is_stroke)

delete_row = resampled_train_df[resampled_train_df["STRK"]==0].index
delete_row = delete_row.values[:1294]
resampled_train_df= resampled_train_df.drop(delete_row)
total = len(resampled_train_df['STRK'])
is_stroke = list(resampled_train_df['STRK']).count(1)
print("NO STROKE: ",total-is_stroke)
print("STROKE: ",is_stroke)

x_train = resampled_train_df.drop(['STRK'],axis=1)
y_train = resampled_train_df['STRK']

xtrain, x_test,ytrain, y_test = train_test_split(x_train,y_train,test_size=0.0000000000001,shuffle=True,random_state=30)

NO STROKE:  5180
STROKE:  260
NO STROKE:  3886
STROKE:  260


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score,confusion_matrix
from sklearn.metrics import fbeta_score, make_scorer

w = [ {0:1.0,1:4.0}]
lr_w=LogisticRegression(max_iter=700000)
score_f1_beta = make_scorer(fbeta_score, beta=2)
grid_values_w = {'class_weight':w, 'C':[ 10000]}
grid_lr_acc_w = GridSearchCV(lr_w, param_grid = grid_values_w, cv=5, verbose=10, n_jobs=-1,  scoring = score_f1_beta )
grid_lr_acc_w.fit(xtrain, ytrain)
grid_lr_acc_w.best_params_
import pickle
filename = 'LOG25%.sav'
pickle.dump(grid_lr_acc_w, open(filename, 'wb'))
print(f'Best score: {grid_lr_acc_w.best_score_} with param: {grid_lr_acc_w.best_params_}')
#Predict values based on new parameters

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best score: 0.8992798691491674 with param: {'C': 10000, 'class_weight': {0: 1.0, 1: 4.0}}


In [6]:
print(f'Best score: {grid_lr_acc_w.best_score_} with param: {grid_lr_acc_w.best_params_}')
lrgpytest_w= grid_lr_acc_w.predict(xtest)
print('accuracy_score\n',accuracy_score(ytest,lrgpytest_w))
print('precision_score\n',precision_score(ytest,lrgpytest_w))
print('recall_score\n',recall_score(ytest,lrgpytest_w))
print('f1_score beta\n',fbeta_score(ytest,lrgpytest_w, beta=2.0))
grid_lr_acc_acc_sc=accuracy_score(ytest,lrgpytest_w)
grid_lr_acc_pr_sc=precision_score(ytest,lrgpytest_w)
grid_lr_acc_rec_sc=recall_score(ytest,lrgpytest_w)
grid_lr_acc_f1_sc=f1_score(ytest,lrgpytest_w)
confusion_matrix(ytest,lrgpytest_w)

Best score: 0.8992798691491674 with param: {'C': 10000, 'class_weight': {0: 1.0, 1: 4.0}}
accuracy_score
 0.9897058823529412
precision_score
 0.8852459016393442
recall_score
 0.8852459016393442
f1_score beta
 0.8852459016393442


array([[1292,    7],
       [   7,   54]])

In [7]:
from sklearn.svm import SVC
from sklearn.metrics import fbeta_score, make_scorer
w = [ {0:1.0,1:4.0}]
lsvc_params1  = {
    "kernel": ['linear'],
    "C" : [1000],
    'class_weight':w,
    'degree': [2]
}
score_f1_beta = make_scorer(fbeta_score, beta=2)
lsvc1 =SVC()
lsvc_tune1 = GridSearchCV(estimator = lsvc1, param_grid = lsvc_params1, verbose=10, scoring = score_f1_beta, cv = 5, return_train_score = True)
lsvc_tune1.fit(xtrain, ytrain)
lsvc_tune1.best_params_

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START C=1000, class_weight={0: 1.0, 1: 4.0}, degree=2, kernel=linear
[CV 1/5; 1/1] END C=1000, class_weight={0: 1.0, 1: 4.0}, degree=2, kernel=linear; total time=  11.7s
[CV 2/5; 1/1] START C=1000, class_weight={0: 1.0, 1: 4.0}, degree=2, kernel=linear
[CV 2/5; 1/1] END C=1000, class_weight={0: 1.0, 1: 4.0}, degree=2, kernel=linear; total time=  19.6s
[CV 3/5; 1/1] START C=1000, class_weight={0: 1.0, 1: 4.0}, degree=2, kernel=linear
[CV 3/5; 1/1] END C=1000, class_weight={0: 1.0, 1: 4.0}, degree=2, kernel=linear; total time=   9.0s
[CV 4/5; 1/1] START C=1000, class_weight={0: 1.0, 1: 4.0}, degree=2, kernel=linear
[CV 4/5; 1/1] END C=1000, class_weight={0: 1.0, 1: 4.0}, degree=2, kernel=linear; total time=  21.0s
[CV 5/5; 1/1] START C=1000, class_weight={0: 1.0, 1: 4.0}, degree=2, kernel=linear
[CV 5/5; 1/1] END C=1000, class_weight={0: 1.0, 1: 4.0}, degree=2, kernel=linear; total time=  23.1s


{'C': 1000, 'class_weight': {0: 1.0, 1: 4.0}, 'degree': 2, 'kernel': 'linear'}

In [8]:
import pickle
filename = 'SVC25%.sav'
pickle.dump(lsvc_tune1, open(filename, 'wb'))
print(f'Best score: {lsvc_tune1.best_score_} with param: {lsvc_tune1.best_params_}')
#Predict values based on new parameters

Best score: 0.9247711638738675 with param: {'C': 1000, 'class_weight': {0: 1.0, 1: 4.0}, 'degree': 2, 'kernel': 'linear'}


In [9]:
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score,confusion_matrix
scv_1_preds1 = lsvc_tune1.predict(xtest)
print('accuracy_score\n',accuracy_score(ytest,scv_1_preds1))
print('precision_score\n',precision_score(ytest,scv_1_preds1))
print('recall_score\n',recall_score(ytest,scv_1_preds1))
print('f1_score beta\n',fbeta_score(ytest,scv_1_preds1, beta=2.0))
grid_lr_acc_acc_sc=accuracy_score(ytest,scv_1_preds1)
grid_lr_acc_pr_sc=precision_score(ytest,scv_1_preds1)
grid_lr_acc_rec_sc=recall_score(ytest,scv_1_preds1)
grid_lr_acc_f1_sc=f1_score(ytest,scv_1_preds1)
confusion_matrix(ytest,scv_1_preds1)

accuracy_score
 0.9926470588235294
precision_score
 0.9180327868852459
recall_score
 0.9180327868852459
f1_score beta
 0.9180327868852459


array([[1294,    5],
       [   5,   56]])

In [10]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

ada_1 = AdaBoostClassifier()
ada_1_params  = {
     "n_estimators":[2500],
    "learning_rate" : [1.5]
}
score_f1_beta = make_scorer(fbeta_score, beta=2)
ada_1_tune = GridSearchCV(estimator = ada_1, param_grid = ada_1_params, scoring = score_f1_beta, cv = 5, return_train_score = True, verbose = 10)
ada_1_tune.fit(xtrain, ytrain)
ada_1_tune.best_params_

import pickle
filename = 'ADA25%.sav'
pickle.dump(ada_1_tune, open(filename, 'wb'))
print(f'Best score: {ada_1_tune.best_score_} with param: {ada_1_tune.best_params_}')
#Predict values based on new parameters

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START learning_rate=1.5, n_estimators=2500........................
[CV 1/5; 1/1] END ......learning_rate=1.5, n_estimators=2500; total time=  24.3s
[CV 2/5; 1/1] START learning_rate=1.5, n_estimators=2500........................
[CV 2/5; 1/1] END ......learning_rate=1.5, n_estimators=2500; total time=  24.4s
[CV 3/5; 1/1] START learning_rate=1.5, n_estimators=2500........................
[CV 3/5; 1/1] END ......learning_rate=1.5, n_estimators=2500; total time=  24.3s
[CV 4/5; 1/1] START learning_rate=1.5, n_estimators=2500........................
[CV 4/5; 1/1] END ......learning_rate=1.5, n_estimators=2500; total time=  24.4s
[CV 5/5; 1/1] START learning_rate=1.5, n_estimators=2500........................
[CV 5/5; 1/1] END ......learning_rate=1.5, n_estimators=2500; total time=  24.3s
Best score: 0.768026405401119 with param: {'learning_rate': 1.5, 'n_estimators': 2500}


In [12]:
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score,confusion_matrix
ada_1_preds = ada_1_tune.predict(xtest)
print('accuracy_score\n',accuracy_score(ytest,ada_1_preds))
print('precision_score\n',precision_score(ytest,ada_1_preds))
print('recall_score\n',recall_score(ytest,ada_1_preds))
print('f1_score beta\n',fbeta_score(ytest,ada_1_preds, beta=2.0))
grid_lr_acc_acc_sc=accuracy_score(ytest,ada_1_preds)
grid_lr_acc_pr_sc=precision_score(ytest,ada_1_preds)
grid_lr_acc_rec_sc=recall_score(ytest,ada_1_preds)
grid_lr_acc_f1_sc=f1_score(ytest,ada_1_preds)
confusion_matrix(ytest,ada_1_preds)

accuracy_score
 0.9845588235294118
precision_score
 0.8703703703703703
recall_score
 0.7704918032786885
f1_score beta
 0.7885906040268457


array([[1292,    7],
       [  14,   47]])

In [13]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
rfc_1 = RandomForestClassifier()
rfc_1_params  = {
    "n_estimators":[3000],
    "criterion" : ['gini'],
}
score_f1_beta = make_scorer(fbeta_score, beta=2)
rfc_1_tune = GridSearchCV(estimator = rfc_1, param_grid = rfc_1_params, cv = 5, scoring=score_f1_beta, return_train_score = True, verbose = 10)
rfc_1_tune.fit(xtrain, ytrain)
rfc_1_tune.best_params_
filename = 'RFC25.sav'
pickle.dump(rfc_1_tune, open(filename, 'wb'))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START criterion=gini, n_estimators=3000...........................
[CV 1/5; 1/1] END .........criterion=gini, n_estimators=3000; total time=  17.9s
[CV 2/5; 1/1] START criterion=gini, n_estimators=3000...........................
[CV 2/5; 1/1] END .........criterion=gini, n_estimators=3000; total time=  17.9s
[CV 3/5; 1/1] START criterion=gini, n_estimators=3000...........................
[CV 3/5; 1/1] END .........criterion=gini, n_estimators=3000; total time=  17.6s
[CV 4/5; 1/1] START criterion=gini, n_estimators=3000...........................
[CV 4/5; 1/1] END .........criterion=gini, n_estimators=3000; total time=  18.1s
[CV 5/5; 1/1] START criterion=gini, n_estimators=3000...........................
[CV 5/5; 1/1] END .........criterion=gini, n_estimators=3000; total time=  18.0s


In [14]:
rfc_1_preds = rfc_1_tune.predict(xtest)
print('accuracy_score\n',accuracy_score(ytest,rfc_1_preds))
print('precision_score\n',precision_score(ytest,rfc_1_preds))
print('recall_score\n',recall_score(ytest,rfc_1_preds))
print('f1_score beta\n',fbeta_score(ytest,rfc_1_preds, beta=2.0))
grid_lr_acc_acc_sc=accuracy_score(ytest,rfc_1_preds)
grid_lr_acc_pr_sc=precision_score(ytest,rfc_1_preds)
grid_lr_acc_rec_sc=recall_score(ytest,rfc_1_preds)
grid_lr_acc_f1_sc=f1_score(ytest,rfc_1_preds)
confusion_matrix(ytest,rfc_1_preds)

accuracy_score
 0.9772058823529411
precision_score
 1.0
recall_score
 0.4918032786885246
f1_score beta
 0.5474452554744526


array([[1299,    0],
       [  31,   30]])

In [15]:
from sklearn.neural_network import MLPClassifier
score_f1_beta = make_scorer(fbeta_score, beta=2)
parameter_space = {
    'activation': ['tanh'],
    'solver': [ 'lbfgs'],
    'alpha': [.001],
    'learning_rate': ['constant'],
}
score_f1_beta = make_scorer(fbeta_score, beta=2)
mlp_gs = MLPClassifier(max_iter=10000)
clf = GridSearchCV(mlp_gs, parameter_space, cv=5, scoring=score_f1_beta, verbose=10)
clf.fit(xtrain, ytrain)
clf.best_params_
import pickle
filename = 'MLP25%.sav'
pickle.dump(clf, open(filename, 'wb'))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5; 1/1] START activation=tanh, alpha=0.001, learning_rate=constant, solver=lbfgs
[CV 1/5; 1/1] END activation=tanh, alpha=0.001, learning_rate=constant, solver=lbfgs; total time=   1.7s
[CV 2/5; 1/1] START activation=tanh, alpha=0.001, learning_rate=constant, solver=lbfgs
[CV 2/5; 1/1] END activation=tanh, alpha=0.001, learning_rate=constant, solver=lbfgs; total time=   1.8s
[CV 3/5; 1/1] START activation=tanh, alpha=0.001, learning_rate=constant, solver=lbfgs
[CV 3/5; 1/1] END activation=tanh, alpha=0.001, learning_rate=constant, solver=lbfgs; total time=   1.8s
[CV 4/5; 1/1] START activation=tanh, alpha=0.001, learning_rate=constant, solver=lbfgs
[CV 4/5; 1/1] END activation=tanh, alpha=0.001, learning_rate=constant, solver=lbfgs; total time=   1.6s
[CV 5/5; 1/1] START activation=tanh, alpha=0.001, learning_rate=constant, solver=lbfgs
[CV 5/5; 1/1] END activation=tanh, alpha=0.001, learning_rate=constant, solver=lbfgs; 

In [16]:
clf_preds = clf.predict(xtest)
print('accuracy_score\n',accuracy_score(ytest,clf_preds))
print('precision_score\n',precision_score(ytest,clf_preds))
print('recall_score\n',recall_score(ytest,clf_preds))
print('f1_score beta\n',fbeta_score(ytest,clf_preds, beta=2.0))
grid_lr_acc_acc_sc=accuracy_score(ytest,clf_preds)
grid_lr_acc_pr_sc=precision_score(ytest,clf_preds)
grid_lr_acc_rec_sc=recall_score(ytest,clf_preds)
grid_lr_acc_f1_sc=f1_score(ytest,clf_preds)
confusion_matrix(ytest,clf_preds)

accuracy_score
 0.986764705882353
precision_score
 0.864406779661017
recall_score
 0.8360655737704918
f1_score beta
 0.8415841584158417


array([[1291,    8],
       [  10,   51]])

In [17]:
from sklearn.metrics import roc_auc_score
print("AUC_ROC log: ", roc_auc_score(ytest, lrgpytest_w))
print("AUC_ROC ada: ", roc_auc_score(ytest, ada_1_preds))
print("AUC_ROC svc: ", roc_auc_score(ytest, scv_1_preds1))
print("AUC_ROC rfc: ", roc_auc_score(ytest, rfc_1_preds))
print("AUC_ROC mlp: ", roc_auc_score(ytest,clf_preds))

AUC_ROC log:  0.9399285705271394
AUC_ROC ada:  0.8825515213468116
AUC_ROC svc:  0.9570918360908139
AUC_ROC rfc:  0.7459016393442623
AUC_ROC mlp:  0.9149534951223515
